# Because DDI consists of entities that are split up across different parts of the sentence, their ending token is the same in the labeling, which causes problems for piecewise splitting of the sentence

In [1]:
%load_ext autoreload

In [41]:
%autoreload

import os, random, pandas as pd, numpy as np
import pickle
import sys
import ast
sys.path.append('../../../')
from relation_extraction.data import utils
import nltk
from ast import literal_eval
import itertools
RESOURCE_PATH = "/data/medg/misc/geeticka/relation_extraction/ddi"
outdir = 'pre-processed/original/piecewise-pool/'
def res(path): return os.path.join(RESOURCE_PATH, path)
from relation_extraction.data.converters.converter_ddi import read_dataframe, \
combine, flatten_list_of_tuples, rev_relation_dict

In [42]:
if not os.path.exists(res(outdir)):
    os.mkdir(res(outdir))

In [5]:
df_train_drugbank = read_dataframe(res('pre-processed/original/' + 'train_drugbank_original.csv'))

In [6]:
df_train_medline = read_dataframe(res('pre-processed/original/' + 'train_medline_original.csv'))

In [7]:
df_test_drugbank = read_dataframe(res('pre-processed/original/' + 'test_drugbank_original.csv'))

In [8]:
df_test_medline = read_dataframe(res('pre-processed/original/' + 'test_medline_original.csv'))

In [43]:
def write_into_txt(df, directory):
    print("Unique relations: \t", df['relation_type'].unique())
    null_row = df[df["relation_type"].isnull()]
    if null_row.empty:
        idx_null_row = None
    else:
        idx_null_row = null_row.index.values[0]
    with open(directory, 'w') as outfile:
        for i in range(0, len(df)):
            if idx_null_row is not None and i == idx_null_row:
                continue
            row = df.iloc[i]
            relation = rev_relation_dict[row.relation_type]
            metadata = row.metadata
            e1 = flatten_list_of_tuples(metadata['e1']['word_index'])
            e2 = flatten_list_of_tuples(metadata['e2']['word_index'])
            common = list(set(e1).intersection(set(e2)))
            e1 = [x for x in e1 if x not in common]
            e2 = [x for x in e2 if x not in common]
            e1 = sorted(e1)
            e2 = sorted(e2)
            tokenized_sentence = row.tokenized_sentence
            outfile.write(str(relation) + " " + str(e1[0]) + " " + str(e1[-1]) + " " + 
                          str(e2[0]) + " " + str(e2[-1]) + " " + tokenized_sentence + "\n")
        outfile.close()

In [44]:
write_into_txt(df_train_drugbank, res(outdir + 'train_drugbank_original.txt'))

Unique relations: 	 ['effect' 'none' 'advise' 'mechanism' 'int']


In [45]:
write_into_txt(df_train_medline, res(outdir + 'train_medline_original.txt'))

Unique relations: 	 ['none' 'mechanism' 'effect' 'advise' 'int']


In [46]:
write_into_txt(df_test_drugbank, res(outdir + 'test_drugbank_original.txt'))

Unique relations: 	 ['int' 'effect' 'none' 'mechanism' 'advise']


In [47]:
write_into_txt(df_test_medline, res(outdir + 'test_medline_original.txt'))

Unique relations: 	 ['none' 'effect' 'mechanism' 'advise' 'int']


In [48]:
combine(res, outdir, 'train_medline_original', 'train_drugbank_original', 'train_original.txt')

In [49]:
combine(res, outdir, 'test_medline_original', 'test_drugbank_original', 'test_original.txt')